## DIA 038: Implementación de A/B Testing para la API

En este día se implementa un sistema de A/B testing para el endpoint de predicción de la API. La idea es distribuir aleatoriamente las solicitudes entre dos versiones de la API (v1 y v2) para comparar su rendimiento y efectividad. Esto permite recolectar datos y feedback que ayudarán a mejorar el modelo de predicción de forma continua. El endpoint de A/B testing selecciona de manera aleatoria la versión a utilizar para cada solicitud y añade un indicador en la respuesta para identificar la versión empleada.

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
import os
import logging
from datetime import datetime
import time
import threading
import json
import random
from functools import wraps

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos (simplificados)
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Para este ejemplo, se asume que las credenciales son correctas.
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación simple: retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación mejorada: retorna dígito 7 con 95% de probabilidad y un mensaje adicional
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Nuevo Endpoint: A/B Testing para la Predicción
# ---------------------------
ab_blueprint = Blueprint('api_ab', __name__)

@ab_blueprint.route('/predict', methods=['POST'])
@jwt_required()
def predict_ab():
    """
    Endpoint de A/B Testing para la predicción.
    Selecciona aleatoriamente entre la versión v1 y v2 del endpoint de predicción.
    """
    version = random.choice(['v1', 'v2'])
    if version == 'v1':
        response = predict_v1()  # Llama a la función de predicción de v1
    else:
        response = predict_v2()  # Llama a la función de predicción de v2
    # Añadir el identificador de la versión A/B en la respuesta
    data = response.get_json()
    data['ab_version'] = version
    logger.info(f"A/B Testing: Solicitud enrutada a la versión {version}.")
    return jsonify(data), response.status_code

app.register_blueprint(ab_blueprint, url_prefix='/api/ab')

# ---------------------------
# Endpoint de Feedback (simplificado)
# ---------------------------
@app.route('/feedback', methods=['POST'])
@jwt_required()
def submit_feedback():
    current_user = get_jwt_identity()
    data = request.get_json()
    prediction = data.get('prediction')
    correct = data.get('correct')
    comment = data.get('comment', "")
    if prediction is None or correct is None:
        logger.warning(f"Feedback fallido: Datos incompletos del usuario '{current_user}'.")
        return jsonify({"msg": "Prediction and correct flag required"}), 400
    feedback = Feedback(
        username=current_user,
        prediction=prediction,
        correct=correct,
        comment=comment
    )
    db.session.add(feedback)
    db.session.commit()
    logger.info(f"Feedback registrado por '{current_user}': {feedback.to_dict()}")
    # Aquí se podrían emitir notificaciones a administradores
    return jsonify({"msg": "Feedback submitted successfully"}), 201

# ---------------------------
# Endpoint de Retraining Basado en Feedback (simplificado)
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
def retrain_model_endpoint():
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        time.sleep(10)  # Simula el tiempo de retraining
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Endpoint de Reporte Automatizado (simplificado)
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
def generate_report():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
A/B Testing para la Predicción:

Se crea un nuevo Blueprint (api_ab) que define el endpoint /predict para A/B testing.
El endpoint selecciona aleatoriamente entre la versión v1 y v2 usando random.choice y llama a la función correspondiente.
Se añade un campo ab_version en la respuesta para indicar qué versión se utilizó.
Integración con el Resto de la API:

Se registran Blueprints para v1, v2 y A/B testing, permitiendo que los clientes accedan a la versión que deseen.
Se mantiene la estructura existente de endpoints para feedback, retraining y reportes.
Ejecución:

La aplicación se ejecuta con Flask-SocketIO para soportar tanto HTTP como WebSocket, y se utiliza socketio.run(app, debug=True).
Este código permite realizar A/B testing en el endpoint de predicción, facilitando la comparación entre dos versiones del modelo y ayudando a recoger datos para optimizar la precisión y el rendimiento del sistema.

Resumen en Viñetas del Día 38: Implementación de A/B Testing para la API

Objetivo Principal:

Evaluar y comparar el rendimiento de dos versiones del endpoint de predicción (v1 y v2).
Implementación del Endpoint de A/B Testing:

Se crea un nuevo Blueprint (api_ab) con un endpoint /predict que decide aleatoriamente entre v1 y v2.
Utiliza random.choice para seleccionar la versión a utilizar.
Añade un campo ab_version en la respuesta para indicar la versión empleada.
Beneficios:

Permite recolectar datos comparativos de ambas versiones del modelo.
Facilita la toma de decisiones para futuras mejoras basadas en el rendimiento real.
No afecta a los usuarios, ya que la distribución es aleatoria y transparente.
Integración:

Se integra con el sistema de autenticación y se mantiene la estructura de la API versionada (v1 y v2).
El endpoint de A/B testing reutiliza la lógica existente de los endpoints de predicción.
